In [1]:
# importing the necessary packages
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="31ca9408c9f14c5e83c94b8a2f10c5ad" 
secret = "7b582ea9c6b44851be183da18c77acd2"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
import pandas as pd
from pandas import ExcelFile
from pandas import ExcelWriter

tracks = pd.read_excel('regional-global-weekly-latest - xlsv.xlsx')

print("Column headings:")
print(tracks.columns)

Column headings:
Index(['Position', 'TrackName', 'Artist', 'Streams', 'URL', 'URI'], dtype='object')


In [7]:
df_tracks = pd.DataFrame({'artist_name':tracks.Artist,'track_name':tracks.TrackName,'track_id':tracks.URI,'popularity':tracks.Position})
print(df_tracks.shape)
df_tracks.head()

(200, 4)


,artist_name,track_name,track_id,popularity
0,The Weeknd,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,1
1,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,2
2,SAINt JHN,Roses - Imanbek Remix,24Yi9hE78yPEbZ4kxyoXAI,3
3,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,4
4,Dua Lipa,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,5


In [9]:
grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

0

In [10]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [11]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [12]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()


artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [13]:
#again measuring the time
import timeit
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.5588975000000005


In [14]:
print('number of elements in the track_id list:', len(rows))


number of elements in the track_id list: 200


In [15]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (200, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
1,0.834,0.454,1,-9.750,0,0.2010,0.32100,0.000006,0.1140,0.837,81.618,audio_features,127QTOFJsJQp5LbJbu3A1y,spotify:track:127QTOFJsJQp5LbJbu3A1y,https://api.spotify.com/v1/tracks/127QTOFJsJQp...,https://api.spotify.com/v1/audio-analysis/127Q...,247059,4
2,0.770,0.724,8,-5.484,1,0.0495,0.01670,0.010700,0.3530,0.898,121.975,audio_features,24Yi9hE78yPEbZ4kxyoXAI,spotify:track:24Yi9hE78yPEbZ4kxyoXAI,https://api.spotify.com/v1/tracks/24Yi9hE78yPE...,https://api.spotify.com/v1/audio-analysis/24Yi...,176840,4
3,0.726,0.431,8,-8.765,0,0.1350,0.73100,0.000000,0.6960,0.348,144.026,audio_features,7eJMfftS33KTjuF7lTsMCx,spotify:track:7eJMfftS33KTjuF7lTsMCx,https://api.spotify.com/v1/tracks/7eJMfftS33KT...,https://api.spotify.com/v1/audio-analysis/7eJM...,173333,4
4,0.794,0.793,11,-4.521,0,0.0842,0.01250,0.000000,0.0952,0.677,123.941,audio_features,3PfIrDoz19wz7qK7tYeu62,spotify:track:3PfIrDoz19wz7qK7tYeu62,https://api.spotify.com/v1/tracks/3PfIrDoz19wz...,https://api.spotify.com/v1/audio-analysis/3PfI...,183290,4


In [16]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(200, 14)

In [17]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (200, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,The Weeknd,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,1,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,200040,4
1,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,2,0.834,0.454,1,-9.750,0,0.2010,0.32100,0.000006,0.1140,0.837,81.618,247059,4
2,SAINt JHN,Roses - Imanbek Remix,24Yi9hE78yPEbZ4kxyoXAI,3,0.770,0.724,8,-5.484,1,0.0495,0.01670,0.010700,0.3530,0.898,121.975,176840,4
3,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,4,0.726,0.431,8,-8.765,0,0.1350,0.73100,0.000000,0.6960,0.348,144.026,173333,4
4,Dua Lipa,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,5,0.794,0.793,11,-4.521,0,0.0842,0.01250,0.000000,0.0952,0.677,123.941,183290,4


In [18]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [19]:
df.to_csv('SpotifyAudioFeatures23042020.csv')